In [1]:
import pandas as pd
#from decimal import Decimal
from re import sub

In [2]:
loc = '/home/tbrownex/data/investing/Fisher/transactions/'

In [14]:
def fixDate(df):
    '''
    Input date column has some that look like "02/16/2024 as of 02/15/2024"
    In those cases, strip off the beginning stuff and return 02/15/2024 so
    the column can be formatted as a Date
    '''
    def findAsOf(x):
        pos = x.find('as of')
        pos += 5
        return x[pos:]
    ok = df[~df['Date'].str.contains('as')]
    notOK = df[df['Date'].str.contains('as')]
    assert len(ok)+len(notOK)==len(df), 'invalid lengths'
    notOK['Date'] = notOK['Date'].apply(lambda x: findAsOf(x))
    both = pd.concat([ok, notOK])
    # Now make it a proper date field
    both['date'] = both['Date'].apply(lambda x: pd.to_datetime(x))
    return both

In [120]:
#beneficiary = pd.read_csv(loc+'beneficiary_2023.csv')
#pension = pd.read_csv(loc+'pension_2023.csv')
#rollover = pd.read_csv(loc+'rollover_2023.csv')
roth = pd.read_csv(loc+'roth_transactions.csv')

In [122]:
js = roth[roth['Action'] == 'Journaled Shares']

In [126]:
js.loc[71]['Description']

'TDA TRAN - CASH ALTERNATIVES REDEMPTION (MMDA13) BANK SWEEP WITHDRAWAL'

In [117]:
'''beneficiary['fund'] = 'beneficiary'
pension['fund'] = 'pension'
rollover['fund'] = 'rollover'''
roth['fund'] = 'roth'

In [118]:
roth['amount'] = roth['Amount'].replace('[\$,]', '', regex=True).astype(float)
roth = fixDate(roth)
roth = roth[['fund', 'date', 'Action', 'amount']]
roth.columns =['fund', 'date', 'action', 'amount']
roth = roth[roth['action'].isin(['MoneyLink Transfer', 'Advisor Fee'])]

/tmp/ipykernel_1482/3637582337.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notOK['Date'] = notOK['Date'].apply(lambda x: findAsOf(x))


In [119]:
roth

,fund,date,action,amount
3,roth,2024-04-18,Advisor Fee,-110.54
9,roth,2024-01-19,Advisor Fee,-102.23
16,roth,2023-11-10,Advisor Fee,-92.59


In [6]:
df = pd.concat([beneficiary, pension, rollover, roth])
# Remove $ sign from account values
df['amount'] = df['Amount'].replace('[\$,]', '', regex=True).astype(float)
# Some of these records have an "as of" date instead of a normal date
df = fixDate(df)
df = df[['fund', 'date', 'Action', 'amount']]
df.columns =['fund', 'date', 'action', 'amount']

/tmp/ipykernel_1444/3637582337.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notOK['Date'] = notOK['Date'].apply(lambda x: findAsOf(x))


In [7]:
# Only interested in deposits, withdrawals and advisor fees
df = df[df['action'].isin(['MoneyLink Transfer', 'Advisor Fee'])]
df.reset_index(drop=True, inplace=True)

In [8]:
df[df['action']=='MoneyLink Transfer']

,fund,date,action,amount
2,beneficiary,2023-08-31,MoneyLink Transfer,-40000.0
3,beneficiary,2023-06-20,MoneyLink Transfer,-15000.0
4,beneficiary,2023-03-13,MoneyLink Transfer,-15000.0
5,beneficiary,2023-02-24,MoneyLink Transfer,-15000.0
6,beneficiary,2022-11-30,MoneyLink Transfer,-30000.0
7,beneficiary,2022-06-29,MoneyLink Transfer,-70000.0
14,pension,2023-09-14,MoneyLink Transfer,60000.0


In [9]:
df.sort_values('date')[:]

,fund,date,action,amount
7,beneficiary,2022-06-29,MoneyLink Transfer,-70000.00
6,beneficiary,2022-11-30,MoneyLink Transfer,-30000.00
5,beneficiary,2023-02-24,MoneyLink Transfer,-15000.00
4,beneficiary,2023-03-13,MoneyLink Transfer,-15000.00
3,beneficiary,2023-06-20,MoneyLink Transfer,-15000.00
2,beneficiary,2023-08-31,MoneyLink Transfer,-40000.00
14,pension,2023-09-14,MoneyLink Transfer,60000.00
1,beneficiary,2023-11-10,Advisor Fee,-620.80
9,pension,2023-11-10,Advisor Fee,-381.92
11,rollover,2023-11-10,Advisor Fee,-5914.99
